In [1]:
cd ..

C:\Users\kevin.rose\Documents\GitHub\cjp-article-tagging\lib


In [2]:
import tagnews
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dense, TimeDistributed
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
import numpy as np
import json
import requests
import keras

Using TensorFlow backend.


In [3]:
glove = tagnews.load_glove('tagnews/data/glove.6B.50d.txt')
ner = tagnews.load_ner_data('tagnews/data/')
ner = pd.concat([ner, pd.DataFrame(glove.loc[ner['word'].str.lower()].values)], axis='columns')
ner.fillna(value=0.0, inplace=True)

data_dim = 50
timesteps = 25 # only during training, testing can take arbitrary length.
num_classes = 2

train_val_split = int(19 * ner.shape[0] / 20.)

ner_train_idxs = range(0, train_val_split - timesteps, timesteps)
x_train = np.array([ner.iloc[i:i+timesteps, 3:].values
                    for i in ner_train_idxs])
y_train = np.array([to_categorical(ner.iloc[i:i+timesteps, 2].values, 2)
                    for i in ner_train_idxs])

ner_val_idxs = range(train_val_split, ner.shape[0] - timesteps, timesteps)
x_val = np.array([ner.iloc[i:i+timesteps, 3:].values
                  for i in ner_val_idxs])
y_val = np.array([to_categorical(ner.iloc[i:i+timesteps, 2].values, 2)
                  for i in ner_val_idxs])

b'Skipping line 281837: expected 25 fields, saw 34\n'
C:\Users\kevin.rose\AppData\Local\Continuum\Anaconda2\envs\cjp\lib\site-packages\numpy\lib\arraysetops.py:463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [4]:
model = Sequential()
model.add(LSTM(32, return_sequences=True, input_shape=(None, data_dim)))
model.add(LSTM(8, return_sequences=True))
model.add(TimeDistributed(Dense(2, activation='softmax')))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['categorical_accuracy'])
print(model.summary(100))

____________________________________________________________________________________________________
Layer (type)                                 Output Shape                            Param #        
lstm_1 (LSTM)                                (None, None, 32)                        10624          
____________________________________________________________________________________________________
lstm_2 (LSTM)                                (None, None, 8)                         1312           
____________________________________________________________________________________________________
time_distributed_1 (TimeDistributed)         (None, None, 2)                         18             
Total params: 11,954
Trainable params: 11,954
Non-trainable params: 0
____________________________________________________________________________________________________
None


In [5]:
checkpointer = ModelCheckpoint(filepath='./tmp/weights-{epoch:02d}.hdf5',
                               monitor='val_categorical_accuracy',
                               verbose=1,
                               save_best_only=True)

class OurAUC(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        # Go to https://geo-extract-tester.herokuapp.com/ and download
        # the validation data (validation.txt).
        with open('validation.txt', encoding='utf-8') as f:
            s = f.read()

        gloved_data = glove.loc[[w for w in s.split('\n') if w]].fillna(0)
        glove_time_size=100
        preds_batched = []
        i = 0
        while gloved_data[i:i+glove_time_size].size:
            preds_batched.append(model.predict(np.expand_dims(gloved_data[i:i+glove_time_size],
                                                              axis=0))[0][:,1])
            i += glove_time_size

        with open('guesses-{epoch:02d}.txt'.format(epoch=epoch), 'w') as f:
            for prob in [p for pred in preds_batched for p in pred]:
                f.write(str(prob) + '\n')

        with open('guesses-{epoch:02d}.txt'.format(epoch=epoch), 'rb') as f:
            url = 'https://geo-extract-tester.herokuapp.com/api/score'
            r = requests.post(url, files={'file': f})
            print('AUC: {:.5f}'.format(json.loads(r.text)['auc']))

our_auc = OurAUC()

In [6]:
model.fit(x_train, y_train,
          epochs=5,
          validation_data=(x_val, y_val),
          callbacks=[checkpointer, our_auc],
          verbose=2)

Train on 39930 samples, validate on 2101 samples
Epoch 1/5
Epoch 00001: val_categorical_accuracy improved from -inf to 0.98226, saving model to ./tmp/weights-01.hdf5
AUC: 0.84700
 - 70s - loss: 0.0988 - categorical_accuracy: 0.9680 - val_loss: 0.0528 - val_categorical_accuracy: 0.9823
Epoch 2/5
Epoch 00002: val_categorical_accuracy improved from 0.98226 to 0.98359, saving model to ./tmp/weights-02.hdf5
AUC: 0.84535
 - 49s - loss: 0.0463 - categorical_accuracy: 0.9837 - val_loss: 0.0451 - val_categorical_accuracy: 0.9836
Epoch 3/5
Epoch 00003: val_categorical_accuracy improved from 0.98359 to 0.98462, saving model to ./tmp/weights-03.hdf5
AUC: 0.85409
 - 53s - loss: 0.0414 - categorical_accuracy: 0.9850 - val_loss: 0.0417 - val_categorical_accuracy: 0.9846
Epoch 4/5
Epoch 00004: val_categorical_accuracy did not improve
AUC: 0.85198
 - 57s - loss: 0.0390 - categorical_accuracy: 0.9855 - val_loss: 0.0405 - val_categorical_accuracy: 0.9846
Epoch 5/5
Epoch 00005: val_categorical_accuracy im

In [7]:
idx = slice(501, 550)
print(pd.concat([ner.iloc[idx, :3].reset_index(drop=True),
                 pd.DataFrame(model.predict(np.expand_dims(ner.iloc[idx, 3:].values, 0))[0][:, 1:],
                              columns=['prob_geloc'])],
                axis='columns'))

# Go to https://geo-extract-tester.herokuapp.com/ and download
# the validation data (validation.txt).
with open('validation.txt', encoding='utf-8') as f:
    s = f.read()

gloved_data = glove.loc[[w for w in s.split('\n') if w]].fillna(0)
glove_time_size=100
preds_batched = []
i = 0
while gloved_data[i:i+glove_time_size].size:
    preds_batched.append(model.predict(np.expand_dims(gloved_data[i:i+glove_time_size], axis=0))[0][:,1])
    i += glove_time_size

with open('guesses.txt', 'w') as f:
    for prob in [p for pred in preds_batched for p in pred]:
        f.write(str(prob) + '\n')

# Now go to https://geo-extract-tester.herokuapp.com/ and upload `guesses.txt` to see how you did!

              word all_tags    tag  prob_geloc
0             said        O  False    0.000316
1               it        O  False    0.000067
2             will        O  False    0.000115
3               go        O  False    0.000067
4            ahead        O  False    0.000316
5             with        O  False    0.000034
6                a        O  False    0.000028
7   reconciliation        O  False    0.000495
8       conference        O  False    0.000674
9               to        O  False    0.000034
10           which        O  False    0.001015
11            more        O  False    0.000032
12            than        O  False    0.000035
13           1,300        O  False    0.000469
14          Somali    B-gpe  False    0.002423
15          elders        O  False    0.000335
16               ,        O  False    0.000028
17        warlords        O  False    0.000712
18             and        O  False    0.000040
19     politicians        O  False    0.000133
20           